In [1]:
import findspark

findspark.init()

In [2]:
from pyspark.sql import SparkSession

# ścieżka do bazy danych hurtowni danych oraz plików
warehouse_location = '/opt/spark/work-dir/lab_07/metastore_db'

# utworzenie sesji Spark, ze wskazaniem włączenia obsługi Hive oraz
# lokalizacją przechowywania hurtowni danych
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Apache SQL and Hive")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .enableHiveSupport()\
        .config("spark.sql.warehouse.dir", warehouse_location)\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/21 19:17:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Apache SQL and Hive>

# 1. Spark i SQL
Spark umożliwia zarejestrowanie obiektu DataFrame jako widoku, co umożliwia korzystanie z niego w sposób bardzo zbliżony do pracy z językiem SQL. Poniżej przykład.

In [3]:
df = spark.read.csv(
    '../lab_06/employee.csv', 
    header=True, 
    inferSchema=True
)

In [4]:
# tworzymy widok tymczasowy w pamięci węzła
df.createOrReplaceTempView("EMPLOYEE_DATA")

In [5]:
# wypisanie tabeli, zwróć uwagę na to, czy stworzona tabela jest tymczasowa czy trwała
spark.catalog.listTables()

24/11/21 19:18:48 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/11/21 19:18:48 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/11/21 19:19:08 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/11/21 19:19:08 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.2
24/11/21 19:19:08 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


[Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [6]:
# pobranie danych jak z tabeli SQL
spark.sql("Select * from EMPLOYEE_DATA limit 4").show()
spark.sql("select firstname from EMPLOYEE_DATA").show(10)

+---+---------+------------+---+-------+
| id|firstname|    lastname|age| salary|
+---+---------+------------+---+-------+
|  1| Wojciech|      Szczaw| 62|8132.22|
|  2| Zbigniew|      Wlotka| 66|8268.47|
|  3|    Agata|Mieczykowski| 37|6116.28|
|  4| Zbigniew|  Wróblewski| 25|9377.25|
+---+---------+------------+---+-------+

+----------+
| firstname|
+----------+
|  Wojciech|
|  Zbigniew|
|     Agata|
|  Zbigniew|
|      Adam|
|     Marek|
|  Wojciech|
|  Zbigniew|
| Katarzyna|
|Aleksandra|
+----------+
only showing top 10 rows



In [7]:
spark.sql("select firstname, count(firstname), avg(salary) from EMPLOYEE_DATA group by firstname").show()

[Stage 6:=============================>                             (1 + 1) / 2]

+----------+----------------+------------------+
| firstname|count(firstname)|       avg(salary)|
+----------+----------------+------------------+
|   Wisława|          200237| 7851.794412820825|
|Mieczysław|          199980| 7848.768358535832|
|     Agata|          199789| 7849.547957244942|
| Krzysztof|          200652| 7850.057565087836|
|     Marek|          199198|7847.2436489824695|
|      Adam|          199742| 7851.878832694189|
| Katarzyna|          199822| 7851.342516789995|
|  Wojciech|          199730| 7849.594330596333|
|  Zbigniew|          200545| 7849.557810566169|
|Aleksandra|          200305| 7849.418310676244|
+----------+----------------+------------------+



In [8]:
rising = 0.1 # 10% podwyżki
spark.sql(f"select firstname, lastname, salary, round(salary + salary * {rising},2) as after_rising from EMPLOYEE_DATA").show(5)

+---------+------------+-------+------------+
|firstname|    lastname| salary|after_rising|
+---------+------------+-------+------------+
| Wojciech|      Szczaw|8132.22|     8945.44|
| Zbigniew|      Wlotka|8268.47|     9095.32|
|    Agata|Mieczykowski|6116.28|     6727.91|
| Zbigniew|  Wróblewski|9377.25|    10314.98|
|     Adam|    Kowalski|6995.95|     7695.55|
+---------+------------+-------+------------+
only showing top 5 rows



# 2. Apache Hive

In [9]:
spark.catalog.currentCatalog()

'spark_catalog'

In [10]:
# dla zrealizowania kolejnych przykładów dokonamy kilku modyfikacji pliku employee
# 1. dodanie kolumny ID - indeksu
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn("ID", monotonically_increasing_id())

In [11]:
df.show(10)

+---+----------+------------+---+-------+
| ID| firstname|    lastname|age| salary|
+---+----------+------------+---+-------+
|  0|  Wojciech|      Szczaw| 62|8132.22|
|  1|  Zbigniew|      Wlotka| 66|8268.47|
|  2|     Agata|Mieczykowski| 37|6116.28|
|  3|  Zbigniew|  Wróblewski| 25|9377.25|
|  4|      Adam|    Kowalski| 30|6995.95|
|  5|     Marek|      Wlotka| 60|8800.12|
|  6|  Wojciech|    Kowalski| 49| 8335.9|
|  7|  Zbigniew|       Pysla| 37|5548.33|
|  8| Katarzyna|Mieczykowski| 58| 7164.0|
|  9|Aleksandra|    Barański| 68|7037.87|
+---+----------+------------+---+-------+
only showing top 10 rows



In [12]:
# dokonamy podziału danych i zapisania w różnych formatach
splits = df.randomSplit(weights=[0.3, 0.7], seed=19)

In [13]:
splits[0].count(), splits[1].count()

(598911, 1401089)

In [ ]:
# to dość dziwne zjawisko niezbyt równego podziału danych jest opisane w artykułach:
# https://medium.com/udemy-engineering/pyspark-under-the-hood-randomsplit-and-sample-inconsistencies-examined-7c6ec62644bc
# oraz
# https://www.geeksforgeeks.org/pyspark-randomsplit-and-sample-methods/

In [14]:
# większa część trafi do nowej tymczasowej tabeli
splits[1].createOrReplaceTempView("EMPLOYEE_DATA_SPLIT_1")

In [15]:
# a mniejsza do plików JSON
splits[0].write.json('employee_data.json', mode='overwrite')

In [16]:
!ls ./employee_data.json/*.json

./employee_data.json/part-00000-6eca5db4-562a-4683-9c46-703b41dda1c2-c000.json
./employee_data.json/part-00001-6eca5db4-562a-4683-9c46-703b41dda1c2-c000.json


In [17]:
# aby móc wykorzystać dane w przykładach ze złączaniem, zapiszemy jeszcze próbkę danych z głównej ramki
# z identyfikatorami oraz dodatkową kolumną z podwyżką
from pyspark.sql.functions import col, lit, round

lucky_guys = spark.sql("select * from EMPLOYEE_DATA").sample(0.01)\
.withColumn('rising', lit('10%')).withColumn('salary after rising', round(col('salary') * 1.1, 2))

In [18]:
# zapisujemy szczęściarzy do oddzielnej tabeli w hurtowni
lucky_guys.write.mode('overwrite').saveAsTable("lucky_employees", format='parquet')

                                                                                24/11/21 19:22:18 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/11/21 19:22:18 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/11/21 19:22:18 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/11/21 19:22:18 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [19]:
spark.catalog.listTables()

[Table(name='lucky_employees', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='EMPLOYEE_DATA_SPLIT_1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

## Złączenie danych z różnych źródeł danych

In [20]:
!ls ./metastore_db/lucky_employees/*.parquet

./metastore_db/lucky_employees/part-00000-06b810b3-4cd5-427a-af22-631c17981769-c000.snappy.parquet
./metastore_db/lucky_employees/part-00001-06b810b3-4cd5-427a-af22-631c17981769-c000.snappy.parquet


In [21]:
# przykład złączania danych na różnych źródłach danych
# zapytanie SQL bezpośrednio na plikach - tutaj zapisanych wcześniej JSON-ach oraz parquet
query = """
SELECT ed.ID, ed.firstname, ed.lastname, ed.salary, lucky.rising, lucky.`salary after rising`
FROM json.`./employee_data.json/` as jtable 
join EMPLOYEE_DATA ed on jtable.ID=ed.ID 
join parquet.`./metastore_db/lucky_employees/` as lucky on ed.ID=lucky.ID
"""
df_from_json = spark.sql(query).show(10)

24/11/21 19:23:11 WARN ObjectStore: Failed to get database json, returning NoSuchObjectException
24/11/21 19:23:11 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException
                                                                                

+------+----------+------------+-------+------+-------------------+
|    ID| firstname|    lastname| salary|rising|salary after rising|
+------+----------+------------+-------+------+-------------------+
|313329|   Wisława|      Wlotka|6660.97|   10%|            7327.07|
|356910|     Marek|        Glut|8337.18|   10%|             9170.9|
|460218|     Agata|        Glut|8289.55|   10%|            9118.51|
|466630| Krzysztof|Mieczykowski|6149.86|   10%|            6764.85|
|484580|     Marek|Mieczykowski|8407.88|   10%|            9248.67|
|554314|     Agata|  Malinowski|7294.92|   10%|            8024.41|
|596462|   Wisława|Mieczykowski|5817.28|   10%|            6399.01|
|607470|Mieczysław|  Wróblewski|8101.26|   10%|            8911.39|
|633612|Aleksandra|    Barański|6274.59|   10%|            6902.05|
|652223|  Wojciech|  Malinowski| 8430.6|   10%|            9273.66|
+------+----------+------------+-------+------+-------------------+
only showing top 10 rows



## Dzielenie danych na wiaderka (ang. buckets) i partycje

In [22]:
# ten przykład pokazuje podział na 16 wiaderek danych bazując na podziale po kolumnie ID (tu używane jest hashowanie)
# dane posortowane są w każdym buckecie po kolumnie salary
# dane zapisywane są do hurtowni Hive, a informacje o zapisanych tam tabelach przechowywane są w
# Hive metastore (domyślnie jest do baza danych Derby)
df.write.bucketBy(16, 'ID').mode('overwrite').sortBy('salary').saveAsTable('employee_id_bucketed')

In [24]:
!ls metastore_db/employee_id_bucketed/*.parquet

metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00000.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00001.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00002.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00003.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00004.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00005.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00006.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00007.c000.snappy.parquet
metastore_db/employee_id_bucketed/part-00000-bd54208b-d0a6-4dc7-aef5-f6d631accb7f_00008.c000.snappy.parquet
metastore_db/employee_id_buc

In [25]:
spark.table('employee_id_bucketed').show(10)

+------+---------+------------+---+-------+
|    ID|firstname|    lastname|age| salary|
+------+---------+------------+---+-------+
|272129|     Adam|      Szczaw| 60|3428.34|
|817327| Wojciech|      Szczaw| 44|3448.94|
|398890|    Marek|  Wróblewski| 28|4009.85|
| 35636|Krzysztof|      Wlotka| 57|4066.73|
| 33375|    Agata|      Wlotka| 56|4070.07|
|  6427|     Adam|Mieczykowski| 32| 4083.1|
|129098| Wojciech|    Barański| 58|4101.87|
| 35177|    Marek|        Glut| 63| 4107.8|
|181166|    Agata|      Szczaw| 67|4205.51|
|484952|     Adam|      Wlotka| 27|4219.78|
+------+---------+------------+---+-------+
only showing top 10 rows



In [26]:
# wypisanie tabeli
spark.catalog.listTables()

[Table(name='employee_id_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='lucky_employees', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='EMPLOYEE_DATA_SPLIT_1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [28]:
# usunięcie tabeli
spark.sql('DROP TABLE employee_id_bucketed')

DataFrame[]

In [29]:
# jeżeli dane, z którymi pracujemy zawierają stosunkowo niewiele różnorodnych wartości w danych kolumnach
# lub filtrowanie i obliczenia często odbywają się na podgrupach danych to lepsze efekty uzyskamy
# poprzez wykorzystanie możliwości partycjonowania tych danych, które to partycjonowanie
# będzie również odzwierciedlone w fizycznej strukturze plików na dysku twardym w hurtowni danych

# zobaczmy przykład poniżej

df.write.partitionBy("lastname").mode('overwrite').saveAsTable("employees_partitioned_lastname")

In [30]:
# dobrym pomysłem jest też określenie ilości bucketów wynikających z danych w konkretnej kolumnie
# i wykorzystanie do podziału
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.bucketBy.html
buckets = spark.sql("select distinct firstname from EMPLOYEE_DATA").count()
buckets

10

In [31]:
# widok danych podzielonych na partycję z punktu widzenia systemu plików
!ls metastore_db/employees_partitioned_lastname

'lastname=Barański'		'lastname=Malinowski'	 'lastname=Wlotka'
'lastname=Brzęczyszczykiewicz'	'lastname=Mieczykowski'  'lastname=Wróblewski'
'lastname=Glut'			'lastname=Pysla'	  _SUCCESS
'lastname=Kowalski'		'lastname=Szczaw'


In [32]:
df.filter(df.lastname == 'Pysla').groupby('lastname').agg({'salary': 'avg'}).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[lastname#19], functions=[avg(salary#21)])
   +- Exchange hashpartitioning(lastname#19, 200), ENSURE_REQUIREMENTS, [plan_id=759]
      +- HashAggregate(keys=[lastname#19], functions=[partial_avg(salary#21)])
         +- Filter (isnotnull(lastname#19) AND (lastname#19 = Pysla))
            +- FileScan csv [lastname#19,salary#21] Batched: false, DataFilters: [isnotnull(lastname#19), (lastname#19 = Pysla)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/lab_06/employee.csv], PartitionFilters: [], PushedFilters: [IsNotNull(lastname), EqualTo(lastname,Pysla)], ReadSchema: struct<lastname:string,salary:double>




In [33]:
%%time
df.filter(df.lastname == 'Pysla').groupby('lastname').agg({'salary': 'avg'}).show(10)

[Stage 43:>                                                         (0 + 2) / 2]

+--------+-----------------+
|lastname|      avg(salary)|
+--------+-----------------+
|   Pysla|7851.037662912827|
+--------+-----------------+

CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 2.21 s


In [34]:
spark.sql("select lastname, avg(salary) from employees_partitioned_lastname where lastname='Pysla' group by lastname").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[lastname#493], functions=[avg(salary#492)])
   +- Exchange hashpartitioning(lastname#493, 200), ENSURE_REQUIREMENTS, [plan_id=825]
      +- HashAggregate(keys=[lastname#493], functions=[partial_avg(salary#492)])
         +- FileScan parquet spark_catalog.default.employees_partitioned_lastname[salary#492,lastname#493] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/lab_07/metastore_db/employees_partitioned_las..., PartitionFilters: [isnotnull(lastname#493), (lastname#493 = Pysla)], PushedFilters: [], ReadSchema: struct<salary:double>




In [35]:
%%time
spark.sql("select lastname, avg(salary) from employees_partitioned_lastname where lastname='Pysla' group by lastname").show(10)
# jak widać operacja wykonała się szybciej

+--------+-----------------+
|lastname|      avg(salary)|
+--------+-----------------+
|   Pysla|7851.037662912827|
+--------+-----------------+

CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 450 ms


In [ ]:
# spark.sparkContext.stop()

# Zadanie 1
Wczytaj plik zamowienia za pomocą Sparka do dowolnego typu danych (RDD, Spark DataFrame) i dokonaj transformacji tak aby:

* naprawić problemy z kodowaniem znaków (replace?) w kolumnie Sprzedawca
* poprawić format danych w kolumnie Utarg
* dodać odpowiednie typy danych
* kolumna idZamowienia powinna być traktowana jako klucz (indeks)

In [40]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import regexp_replace, to_date

schema = StructType([
    StructField("Kraj", StringType(), True),
    StructField("Sprzedawca", StringType(), True),
    StructField("Data_zamowienia", StringType(), True),
    StructField("idZamowienia", IntegerType(), True),
    StructField("Utarg", StringType(), True)
])

data = spark.read.csv("zamowienia.txt", header=True, sep=";", schema=schema)
data.show(3)

+------+----------+---------------+------------+-----------+
|  Kraj|Sprzedawca|Data_zamowienia|idZamowienia|      Utarg|
+------+----------+---------------+------------+-----------+
|Polska|  Kowalski|     16.07.2003|       10248|  440,00 z|
|Polska|  Sowiäski|     10.07.2003|       10249|1 863,40 z|
|Niemcy|   Peacock|     12.07.2003|       10250|1 552,60 z|
+------+----------+---------------+------------+-----------+
only showing top 3 rows



24/11/21 19:33:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt


In [42]:
data = data.withColumn("Sprzedawca", regexp_replace(col("Sprzedawca"), "ä", "a"))
data.show(3)

+------+----------+---------------+------------+-----------+
|  Kraj|Sprzedawca|Data_zamowienia|idZamowienia|      Utarg|
+------+----------+---------------+------------+-----------+
|Polska|  Kowalski|     16.07.2003|       10248|  440,00 z|
|Polska|  Sowiaski|     10.07.2003|       10249|1 863,40 z|
|Niemcy|   Peacock|     12.07.2003|       10250|1 552,60 z|
+------+----------+---------------+------------+-----------+
only showing top 3 rows



24/11/21 19:34:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt


In [43]:
data = data.withColumn("Utarg", regexp_replace(col("Utarg"), r"[^\d,]", "")) \
           .withColumn("Utarg", regexp_replace(col("Utarg"), ",", ".")) \
           .withColumn("Utarg", col("Utarg").cast(FloatType()))
data.show(3)

+------+----------+---------------+------------+------+
|  Kraj|Sprzedawca|Data_zamowienia|idZamowienia| Utarg|
+------+----------+---------------+------------+------+
|Polska|  Kowalski|     16.07.2003|       10248| 440.0|
|Polska|  Sowiaski|     10.07.2003|       10249|1863.4|
|Niemcy|   Peacock|     12.07.2003|       10250|1552.6|
+------+----------+---------------+------------+------+
only showing top 3 rows



24/11/21 19:35:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt


In [45]:
data = data.withColumn("Data_zamowienia", to_date(col("Data_zamowienia"), "dd.MM.yyyy"))
data.show(3)

+------+----------+---------------+------------+------+
|  Kraj|Sprzedawca|Data_zamowienia|idZamowienia| Utarg|
+------+----------+---------------+------------+------+
|Polska|  Kowalski|     2003-07-16|       10248| 440.0|
|Polska|  Sowiaski|     2003-07-10|       10249|1863.4|
|Niemcy|   Peacock|     2003-07-12|       10250|1552.6|
+------+----------+---------------+------------+------+
only showing top 3 rows



24/11/21 19:36:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt


In [46]:
data = data.withColumnRenamed("idZamowienia", "id").withColumn("id", col("id").cast(StringType()))
data.show(3)

+------+----------+---------------+-----+------+
|  Kraj|Sprzedawca|Data_zamowienia|   id| Utarg|
+------+----------+---------------+-----+------+
|Polska|  Kowalski|     2003-07-16|10248| 440.0|
|Polska|  Sowiaski|     2003-07-10|10249|1863.4|
|Niemcy|   Peacock|     2003-07-12|10250|1552.6|
+------+----------+---------------+-----+------+
only showing top 3 rows



24/11/21 19:36:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt


# Zadanie 2
 wykonaj wiaderkowanie danych i wykonaj dowolne zapytanie agregujące na tych danych vs. dane nie podzielone na wiaderka - porównaj czas
* wykonaj partycjonowanie danych i zapisz je w formcie csv (wypróbuj partycjonowanie wg. kraju, nazwiska)
* wykonaj zapytanie agregujące z filtrowanie po kolumnie, której użyłeś/-aś do partycjonowania na danych oryginalnych oraz partycjonowanych i porównaj czas wykonania

In [47]:
from pyspark.sql.functions import sum

output_path_kraj = "output/partycje_kraj"
output_path_sprzedawca = "output/partycje_sprzedawca"

data.write.partitionBy("Kraj").csv(output_path_kraj, header=True, mode="overwrite")
data.write.partitionBy("Sprzedawca").csv(output_path_sprzedawca, header=True, mode="overwrite")

24/11/21 19:41:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt
                                                                                24/11/21 19:41:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt
                                                                                

In [48]:
%%time
data.filter(col("Kraj") == "Polska").groupBy("Kraj").agg(sum("Utarg").alias("Suma_Utargu")).show()

+------+----------------+
|  Kraj|     Suma_Utargu|
+------+----------------+
|Polska|333330.908657074|
+------+----------------+

CPU times: user 4.3 ms, sys: 7.71 ms, total: 12 ms
Wall time: 407 ms


In [49]:
%%time
data.filter(col("Sprzedawca") == "Kowalski").groupBy("Sprzedawca").agg(sum("Utarg").alias("Suma_Utargu")).show()

+----------+-----------------+
|Sprzedawca|      Suma_Utargu|
+----------+-----------------+
|  Kowalski|68792.25023651123|
+----------+-----------------+

CPU times: user 4.61 ms, sys: 7.82 ms, total: 12.4 ms
Wall time: 303 ms


In [52]:
%%time
partitioned_data_kraj = spark.read.csv(output_path_kraj, header=True, inferSchema=True)
partitioned_data.filter(col("Kraj") == "Polska").groupBy("Kraj").agg(sum("Utarg").alias("Suma_Utargu")).show()

+------+-----------------+
|  Kraj|      Suma_Utargu|
+------+-----------------+
|Polska|333330.9099999999|
+------+-----------------+

CPU times: user 2.7 ms, sys: 11.3 ms, total: 14 ms
Wall time: 731 ms


In [53]:
%%time
partitioned_data_sprzedawca = spark.read.csv(output_path_sprzedawca, header=True, inferSchema=True)
partitioned_data.filter(col("Sprzedawca") == "Kowalski").groupBy("Sprzedawca").agg(sum("Utarg").alias("Suma_Utargu")).show()

+----------+-----------+
|Sprzedawca|Suma_Utargu|
+----------+-----------+
|  Kowalski|   68792.25|
+----------+-----------+

CPU times: user 11.2 ms, sys: 518 μs, total: 11.7 ms
Wall time: 1.05 s


# Zadanie 3
Z danych wygeneruj 4 różne podzbiory próbek (wiersze wybrane losowo) i dodaj nową kolumnę w każdym z nich, np. w jednym stwórz kolumnę month wyciągając tylko miesiąc z daty, w drugim wartość netto zamówienia (przyjmując, że vat to 23%), w kolejnym zamień nazwisko na wielkie litery, w kolejnym dodaj kolumnę waluta z wartością PLN.

Następnie zapisz każdy z tych zbiorów tak, że:
* zbiór pierwszy to będzie tymczasowa tabela in-memory Sparka
* zbiór drugi to plik(i) parquet
* zbiór trzeci to plik(i) csv
* zbiór czwarty to plik(i) json

Wykonaj zapytanie złączające jak w przykładzie pobierając dane bezpośrednio z plików i wyświetl idZamowienia, Kraj, Sprzedawcę, Datę, Utarg oraz 4 nowo utworzone kolumny.

In [55]:
from pyspark.sql.functions import col, month, upper

subset1 = data.sample(fraction=0.5, seed=42) \
    .withColumn("month", month(col("Data_zamowienia")))

subset2 = data.sample(fraction=0.5, seed=42) \
    .withColumn("netto", col("Utarg") / 1.23)

subset3 = data.sample(fraction=0.5, seed=42) \
    .withColumn("Sprzedawca", upper(col("Sprzedawca")))

subset4 = data.sample(fraction=0.5, seed=42) \
    .withColumn("waluta", lit("PLN"))

subset1.createOrReplaceTempView("subset1_table")
subset2.write.parquet("subset2.parquet", mode="overwrite")
subset3.write.csv("subset3.csv", mode="overwrite", header=True)
subset4.write.json("subset4.json", mode="overwrite")

24/11/21 19:49:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt
24/11/21 19:49:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt
24/11/21 19:49:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt


In [59]:
from pyspark.sql.functions import expr

subset2_parquet = spark.read.parquet("subset2.parquet")
subset3_csv = spark.read.csv("subset3.csv", header=True, inferSchema=True)
subset4_json = spark.read.json("subset4.json")

subset2_parquet.createOrReplaceTempView("subset2_parquet")
subset3_csv.createOrReplaceTempView("subset3_csv")
subset4_json.createOrReplaceTempView("subset4_json")

query = """
SELECT 
    s1.id, 
    s1.Kraj, 
    s1.Sprzedawca,
    s1.Data_zamowienia,
    s1.Utarg,
    s1.month,
    s2.netto,
    s3.Sprzedawca as Sprzedawca_upper,
    s4.waluta
FROM subset1_table s1
JOIN subset2_parquet s2 ON s1.id = s2.id
JOIN subset3_csv s3 ON s1.id = s3.id
JOIN subset4_json s4 ON s1.id = s4.id
"""

result = spark.sql(query)
result.show()

+-----+------+----------+---------------+-------+-----+------------------+----------------+------+
|   id|  Kraj|Sprzedawca|Data_zamowienia|  Utarg|month|             netto|Sprzedawca_upper|waluta|
+-----+------+----------+---------------+-------+-----+------------------+----------------+------+
|10251|Niemcy| Leverling|     2003-07-15| 654.06|    7| 531.7560955760924|       LEVERLING|   PLN|
|10255|Polska|     Dudek|     2003-07-15| 2490.5|    7|2024.7967479674796|           DUDEK|   PLN|
|10258|Niemcy|   Davolio|     2003-07-23|1614.88|    7|1312.9105730754573|         DAVOLIO|   PLN|
|10260|Niemcy|   Peacock|     2003-07-29|1504.65|    7|1223.2927027756607|         PEACOCK|   PLN|
|10264|Polska|  Sowiaski|     2003-08-23| 695.62|    8| 565.5447114773882|        SOWIASKI|   PLN|
|10266|Niemcy| Leverling|     2003-07-31| 346.56|    7| 281.7560955760925|       LEVERLING|   PLN|
|10268|Niemcy|  Callahan|     2003-08-02| 1101.2|    8| 895.2845131478658|        CALLAHAN|   PLN|
|10273|Nie

24/11/21 19:52:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Kraj, Sprzedawca, Data zamowienia, idZamowienia, Utarg
 Schema: Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg
Expected: Data_zamowienia but found: Data zamowienia
CSV file: file:///opt/spark/work-dir/lab_07/zamowienia.txt
